In [ ]:
"""\nStage B: Direct Preference Optimization (DPO) Training\n=====================================================\n\nTurn the SFT specialist into a Hallucination-Resistant Expert.\n\nInput: dpo_train_data.jsonl (Triplets: Prompt + Chosen + Rejected)\nGoal: Teach the model to statistically prefer factual responses over hallucinations\nMechanism: DPO Loss with KL-Divergence penalty to prevent model drift\n\nKey Difference from Stage A:\n- TWO models loaded in memory:\n  * Active Model: Being trained (weights change)\n  * Reference Model: Frozen copy from Stage A (weights frozen)\n- Loss compares probability of chosen vs rejected responses\n- KL penalty prevents active model from deviating from reference model's style\n\nKey Parameters:\n- Learning Rate: 5e-6 or 1e-6 (much lower than SFT!)\n- Beta: 0.1 (KL-divergence penalty weight)\n- Epochs: 1-3 (similar to SFT)\n\nUsage:\n    python stage_b_dpo_training.py \\\n        --sft_model_path \"./models/sft_specialist/final_model\" \\\n        --num_epochs 2 \\\n        --learning_rate 5e-6 \\\n        --beta 0.1\n\"\"\"\n\nimport os\nimport sys\nimport json\nimport argparse\nimport logging\nfrom pathlib import Path\nfrom dataclasses import dataclass, field\nfrom typing import Optional, Dict, List, Tuple\nimport math\n\nimport torch\nimport torch.nn as nn\nimport torch.nn.functional as F\nfrom torch.utils.data import DataLoader\nimport numpy as np\nfrom tqdm import tqdm\n\nfrom transformers import (\n    AutoModelForCausalLM,\n    AutoTokenizer,\n    get_linear_schedule_with_warmup,\n)\nfrom peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, TaskType\n\nfrom dpo_dataset import DPODataset, DPODataCollator, create_dpo_dataloaders\n\n\nlogging.basicConfig(\n    level=logging.INFO,\n    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'\n)\nlogger = logging.getLogger(__name__)\n\n\n@dataclass\nclass DPOConfig:\n    \"\"\"Configuration for DPO training.\"\"\"\n    \n    # SFT Model (reference model)\n    sft_model_path: str = \"./models/sft_specialist/final_model\"\n    \n    # Data\n    train_data_path: str = \"phase2_data/dpo/train_dpo.jsonl\"\n    val_data_path: str = \"phase2_data/dpo/val_dpo.jsonl\"\n    \n    # Training\n    num_epochs: int = 2\n    batch_size: int = 4  # Smaller batch size due to dual model\n    gradient_accumulation_steps: int = 1\n    learning_rate: float = 5e-6  # Much lower than SFT!\n    warmup_steps: int = 100\n    weight_decay: float = 0.01\n    max_grad_norm: float = 1.0\n    \n    # DPO Specific\n    beta: float = 0.1  # KL-divergence penalty weight\n    label_smoothing: float = 0.0  # Optional label smoothing\n    \n    # LoRA Configuration\n    use_lora: bool = True\n    lora_r: int = 16\n    lora_alpha: int = 32\n    lora_dropout: float = 0.05\n    lora_target_modules: List[str] = field(default_factory=lambda: [\"q_proj\", \"v_proj\"])\n    \n    # Model Configuration\n    max_length: int = 512\n    torch_dtype: str = \"float16\"\n    \n    # Output\n    output_dir: str = \"./models/dpo_hallucination_resistant\"\n    save_steps: int = 100\n    eval_steps: int = 50\n    logging_steps: int = 10\n    save_total_limit: int = 3\n    \n    # Hardware\n    use_8bit: bool = False\n    use_gradient_checkpointing: bool = True\n    device_map: str = \"auto\"\n    \n    # Other\n    seed: int = 42\n\n\nclass DPOTrainer:\n    \"\"\"Trainer for Direct Preference Optimization.\"\"\"\n    \n    def __init__(self, config: DPOConfig):\n        self.config = config\n        self.device = torch.device(\"cuda\" if torch.cuda.is_available() else \"cpu\")\n        logger.info(f\"Using device: {self.device}\")\n        \n        torch.manual_seed(config.seed)\n        np.random.seed(config.seed)\n        \n        # Load tokenizer\n        logger.info(f\"Loading tokenizer from {config.sft_model_path}\")\n        self.tokenizer = AutoTokenizer.from_pretrained(config.sft_model_path)\n        if self.tokenizer.pad_token is None:\n            self.tokenizer.pad_token = self.tokenizer.eos_token\n        \n        # Load models\n        logger.info(\"Loading reference and active models\")\n        self.reference_model = self._load_model(config.sft_model_path, freeze=True)\n        self.model = self._load_model(config.sft_model_path, freeze=False)\n        \n        # Apply LoRA if configured\n        if config.use_lora:\n            logger.info(\"Applying LoRA to active model\")\n            self.model = self._apply_lora(self.model)\n        \n        self.reference_model.to(self.device)\n        self.model.to(self.device)\n        \n        self._log_model_info()\n    \n    def _load_model(self, model_path: str, freeze: bool = False):\n        \"\"\"Load model from SFT checkpoint.\"\"\"\n        torch_dtype = getattr(torch, self.config.torch_dtype.replace(\"torch.\", \"\"))\n        \n        try:\n            # Try to load as LoRA model\n            model = AutoPeftModelForCausalLM.from_pretrained(\n                model_path,\n                torch_dtype=torch_dtype,\n                device_map=self.device,\n                load_in_8bit=self.config.use_8bit,\n            )\n            # Merge if we want the base model\n            if not self.config.use_lora:\n                model = model.merge_and_unload()\n            logger.info(\"Loaded as LoRA model\")\n        except:\n            logger.info(\"Loading as regular model\")\n            model = AutoModelForCausalLM.from_pretrained(\n                model_path,\n                torch_dtype=torch_dtype,\n                device_map=self.device,\n                load_in_8bit=self.config.use_8bit,\n            )\n        \n        if self.config.use_gradient_checkpointing:\n            model.gradient_checkpointing_enable()\n        \n        if freeze:\n            for param in model.parameters():\n                param.requires_grad = False\n        \n        return model\n    \n    def _apply_lora(self, model):\n        \"\"\"Apply LoRA to model.\"\"\"\n        lora_config = LoraConfig(\n            r=self.config.lora_r,\n            lora_alpha=self.config.lora_alpha,\n            lora_dropout=self.config.lora_dropout,\n            bias=\"none\",\n            task_type=TaskType.CAUSAL_LM,\n            target_modules=self.config.lora_target_modules,\n        )\n        \n        model = get_peft_model(model, lora_config)\n        return model\n    \n    def _log_model_info(self):\n        \"\"\"Log model information.\"\"\"\n        total = sum(p.numel() for p in self.model.parameters())\n        trainable = sum(p.numel() for p in self.model.parameters() if p.requires_grad)\n        \n        logger.info(f\"Active model total params: {total:,}\")\n        logger.info(f\"Active model trainable params: {trainable:,}\")\n        logger.info(f\"Trainable: {100 * trainable / total:.2f}%\")\n    \n    def dpo_loss(\n        self,\n        model_logps: torch.Tensor,  # Log probabilities from active model\n        ref_logps: torch.Tensor,    # Log probabilities from reference model\n        chosen: torch.Tensor = None, # Not used in simplifed version\n        rejected: torch.Tensor = None,  # Not used in simplified version\n    ) -> Tuple[torch.Tensor, Dict[str, float]]:\n        \"\"\"\n        Compute DPO loss.\n        \n        DPO Loss = -log(sigmoid(beta * (log_chosen - log_rejected_active - log_chosen_ref + log_rejected_ref)))\n        \n        This encourages:\n        - Active model to assign HIGH probability to chosen\n        - Active model to assign LOW probability to rejected\n        - While staying close to reference model (KL penalty)\n        \"\"\"\n        # Extract probabilities\n        model_chosen_logps = model_logps[:, 0]\n        model_rejected_logps = model_logps[:, 1]\n        ref_chosen_logps = ref_logps[:, 0]\n        ref_rejected_logps = ref_logps[:, 1]\n        \n        # DPO objective: maximize the difference while minimizing KL\n        # log odds ratio\n        log_odds = (\n            (model_chosen_logps - model_rejected_logps) - \n            (ref_chosen_logps - ref_rejected_logps)\n        )\n        \n        # Sigmoid of log odds (preference probability)\n        loss = -F.logsigmoid(self.config.beta * log_odds).mean()\n        \n        # Calculate metrics\n        with torch.no_grad():\n            chosen_preference = (model_chosen_logps > model_rejected_logps).float().mean()\n        \n        metrics = {\n            'loss': loss.item(),\n            'chosen_preference': chosen_preference.item(),\n            'avg_log_odds': log_odds.mean().item(),\n        }\n        \n        return loss, metrics\n    \n    @torch.no_grad()\n    def get_batch_logps(\n        self,\n        input_ids: torch.Tensor,\n        attention_mask: torch.Tensor,\n        model,\n    ) -> torch.Tensor:\n        \"\"\"\n        Compute log probabilities for a batch of sequences.\n        \n        Args:\n            input_ids: (batch_size, seq_len)\n            attention_mask: (batch_size, seq_len)\n            model: The model to compute logps with\n        \n        Returns:\n            Log probabilities for each sequence\n        \"\"\"\n        outputs = model(\n            input_ids=input_ids,\n            attention_mask=attention_mask,\n        )\n        \n        logits = outputs.logits\n        \n        # Shift for next token prediction\n        # We want log prob of token at position i given tokens up to i-1\n        logits = logits[:, :-1, :].contiguous()\n        input_ids_for_loss = input_ids[:, 1:].contiguous()\n        attention_mask_for_loss = attention_mask[:, 1:].contiguous()\n        \n        # Get log probabilities\n        log_probs = F.log_softmax(logits, dim=-1)\n        \n        # Gather log probs for actual tokens\n        batch_size, seq_len = input_ids_for_loss.shape\n        indices = input_ids_for_loss.unsqueeze(-1)\n        selected_log_probs = torch.gather(log_probs, -1, indices).squeeze(-1)\n        \n        # Average over tokens (only non-padded)\n        selected_log_probs = selected_log_probs * attention_mask_for_loss\n        batch_logps = selected_log_probs.sum(dim=1) / attention_mask_for_loss.sum(dim=1).clamp(min=1)\n        \n        return batch_logps\n    \n    def create_dataloaders(self) -> Tuple[DataLoader, DataLoader]:\n        \"\"\"Create train and validation dataloaders.\"\"\"\n        logger.info(\"Creating dataloaders\")\n        \n        train_loader, val_loader = create_dpo_dataloaders(\n            train_jsonl=self.config.train_data_path,\n            val_jsonl=self.config.val_data_path,\n            tokenizer=self.tokenizer,\n            batch_size=self.config.batch_size,\n            max_length=self.config.max_length,\n        )\n        \n        logger.info(f\"Train batches: {len(train_loader)}\")\n        logger.info(f\"Val batches: {len(val_loader)}\")\n        \n        return train_loader, val_loader\n    \n    def train(self, train_loader: DataLoader, val_loader: DataLoader) -> Dict:\n        \"\"\"Main training loop.\"\"\"\n        \n        # Optimizer\n        optimizer = torch.optim.AdamW(\n            self.model.parameters(),\n            lr=self.config.learning_rate,\n            weight_decay=self.config.weight_decay,\n        )\n        \n        # Scheduler\n        total_steps = len(train_loader) * self.config.num_epochs\n        scheduler = get_linear_schedule_with_warmup(\n            optimizer,\n            num_warmup_steps=self.config.warmup_steps,\n            num_training_steps=total_steps,\n        )\n        \n        Path(self.config.output_dir).mkdir(parents=True, exist_ok=True)\n        \n        logger.info(f\"Starting DPO training for {self.config.num_epochs} epochs\")\n        logger.info(f\"Beta (KL weight): {self.config.beta}\")\n        logger.info(f\"Learning rate: {self.config.learning_rate}\")\n        \n        training_stats = {\n            'epoch': [],\n            'train_loss': [],\n            'val_loss': [],\n            'chosen_preference': [],\n            'learning_rate': [],\n        }\n        \n        for epoch in range(self.config.num_epochs):\n            logger.info(f\"\\n{'='*60}\")\n            logger.info(f\"Epoch {epoch + 1}/{self.config.num_epochs}\")\n            logger.info(f\"{'='*60}\")\n            \n            train_loss = self._train_epoch(train_loader, optimizer, scheduler)\n            val_loss, val_preference = self._validate(val_loader)\n            \n            logger.info(f\"Train Loss: {train_loss:.4f}\")\n            logger.info(f\"Val Loss: {val_loss:.4f}\")\n            logger.info(f\"Chosen Preference: {val_preference:.2%}\")\n            logger.info(f\"Learning Rate: {scheduler.get_last_lr()[0]:.2e}\")\n            \n            training_stats['epoch'].append(epoch + 1)\n            training_stats['train_loss'].append(train_loss)\n            training_stats['val_loss'].append(val_loss)\n            training_stats['chosen_preference'].append(val_preference)\n            training_stats['learning_rate'].append(scheduler.get_last_lr()[0])\n            \n            # Save checkpoint\n            checkpoint_path = Path(self.config.output_dir) / f\"checkpoint_epoch_{epoch + 1}\"\n            self._save_checkpoint(checkpoint_path)\n            logger.info(f\"Saved checkpoint to {checkpoint_path}\")\n        \n        logger.info(f\"\\n{'='*60}\")\n        logger.info(\"DPO Training completed!\")\n        logger.info(f\"{'='*60}\\n\")\n        \n        final_path = Path(self.config.output_dir) / \"final_model\"\n        self._save_checkpoint(final_path)\n        logger.info(f\"Saved final model to {final_path}\")\n        \n        stats_path = Path(self.config.output_dir) / \"dpo_training_stats.json\"\n        with open(stats_path, 'w') as f:\n            json.dump(training_stats, f, indent=2)\n        logger.info(f\"Saved training stats to {stats_path}\")\n        \n        return training_stats\n    \n    def _train_epoch(self, train_loader: DataLoader, optimizer, scheduler) -> float:\n        \"\"\"Train for one epoch.\"\"\"\n        self.model.train()\n        total_loss = 0.0\n        total_preference = 0.0\n        \n        progress_bar = tqdm(train_loader, desc=\"Training\")\n        \n        for step, batch in enumerate(progress_bar):\n            chosen_input_ids = batch['chosen_input_ids'].to(self.device)\n            chosen_attention_mask = batch['chosen_attention_mask'].to(self.device)\n            rejected_input_ids = batch['rejected_input_ids'].to(self.device)\n            rejected_attention_mask = batch['rejected_attention_mask'].to(self.device)\n            \n            # Get logps from both models\n            with torch.no_grad():\n                ref_chosen_logps = self.get_batch_logps(\n                    chosen_input_ids,\n                    chosen_attention_mask,\n                    self.reference_model\n                )\n                ref_rejected_logps = self.get_batch_logps(\n                    rejected_input_ids,\n                    rejected_attention_mask,\n                    self.reference_model\n                )\n            \n            model_chosen_logps = self.get_batch_logps(\n                chosen_input_ids,\n                chosen_attention_mask,\n                self.model\n            )\n            model_rejected_logps = self.get_batch_logps(\n                rejected_input_ids,\n                rejected_attention_mask,\n                self.model\n            )\n            \n            # Stack logps\n            model_logps = torch.stack([model_chosen_logps, model_rejected_logps], dim=1)\n            ref_logps = torch.stack([ref_chosen_logps, ref_rejected_logps], dim=1)\n            \n            # Compute DPO loss\n            loss, metrics = self.dpo_loss(model_logps, ref_logps)\n            \n            total_loss += loss.item()\n            total_preference += metrics['chosen_preference']\n            \n            # Backward\n            loss.backward()\n            \n            # Gradient clipping\n            torch.nn.utils.clip_grad_norm_(\n                self.model.parameters(),\n                self.config.max_grad_norm\n            )\n            \n            # Optimizer step\n            optimizer.step()\n            scheduler.step()\n            optimizer.zero_grad()\n            \n            progress_bar.set_postfix({\n                'loss': loss.item(),\n                'pref': f\"{metrics['chosen_preference']:.2%}\"\n            })\n            \n            if (step + 1) % self.config.logging_steps == 0:\n                current_lr = scheduler.get_last_lr()[0]\n                logger.info(\n                    f\"Step {step + 1}: Loss = {loss.item():.4f}, \"\n                    f\"Pref = {metrics['chosen_preference']:.2%}, LR = {current_lr:.2e}\"\n                )\n        \n        avg_loss = total_loss / len(train_loader)\n        avg_preference = total_preference / len(train_loader)\n        logger.info(f\"Epoch avg preference: {avg_preference:.2%}\")\n        \n        return avg_loss\n    \n    def _validate(self, val_loader: DataLoader) -> Tuple[float, float]:\n        \"\"\"Validation loop.\"\"\"\n        self.model.eval()\n        total_loss = 0.0\n        total_preference = 0.0\n        \n        with torch.no_grad():\n            for batch in tqdm(val_loader, desc=\"Validating\"):\n                chosen_input_ids = batch['chosen_input_ids'].to(self.device)\n                chosen_attention_mask = batch['chosen_attention_mask'].to(self.device)\n                rejected_input_ids = batch['rejected_input_ids'].to(self.device)\n                rejected_attention_mask = batch['rejected_attention_mask'].to(self.device)\n                \n                # Get logps\n                ref_chosen_logps = self.get_batch_logps(\n                    chosen_input_ids, chosen_attention_mask, self.reference_model\n                )\n                ref_rejected_logps = self.get_batch_logps(\n                    rejected_input_ids, rejected_attention_mask, self.reference_model\n                )\n                \n                model_chosen_logps = self.get_batch_logps(\n                    chosen_input_ids, chosen_attention_mask, self.model\n                )\n                model_rejected_logps = self.get_batch_logps(\n                    rejected_input_ids, rejected_attention_mask, self.model\n                )\n                \n                model_logps = torch.stack([model_chosen_logps, model_rejected_logps], dim=1)\n                ref_logps = torch.stack([ref_chosen_logps, ref_rejected_logps], dim=1)\n                \n                loss, metrics = self.dpo_loss(model_logps, ref_logps)\n                \n                total_loss += loss.item()\n                total_preference += metrics['chosen_preference']\n        \n        avg_loss = total_loss / len(val_loader)\n        avg_preference = total_preference / len(val_loader)\n        \n        return avg_loss, avg_preference\n    \n    def _save_checkpoint(self, save_path: Path):\n        \"\"\"Save model checkpoint.\"\"\"\n        save_path.mkdir(parents=True, exist_ok=True)\n        \n        self.model.save_pretrained(save_path)\n        self.tokenizer.save_pretrained(save_path)\n        \n        config_dict = {\n            'sft_model': self.config.sft_model_path,\n            'max_length': self.config.max_length,\n            'beta': self.config.beta,\n            'lora_r': self.config.lora_r if self.config.use_lora else None,\n        }\n        config_path = save_path / \"dpo_config.json\"\n        with open(config_path, 'w') as f:\n            json.dump(config_dict, f, indent=2)\n\n\ndef main():\n    \"\"\"Main training function.\"\"\"\n    parser = argparse.ArgumentParser(description=\"Stage B: DPO Training\")\n    \n    parser.add_argument(\n        \"--sft_model_path\",\n        default=\"./models/sft_specialist/final_model\",\n        help=\"Path to SFT model from Stage A\"\n    )\n    \n    parser.add_argument(\n        \"--train_data_path\",\n        default=\"phase2_data/dpo/train_dpo.jsonl\",\n        help=\"Path to training DPO data\"\n    )\n    parser.add_argument(\n        \"--val_data_path\",\n        default=\"phase2_data/dpo/val_dpo.jsonl\",\n        help=\"Path to validation DPO data\"\n    )\n    \n    parser.add_argument(\n        \"--num_epochs\", type=int, default=2,\n        help=\"Number of training epochs\"\n    )\n    parser.add_argument(\n        \"--batch_size\", type=int, default=4,\n        help=\"Batch size (smaller than SFT due to dual model)\"\n    )\n    parser.add_argument(\n        \"--learning_rate\", type=float, default=5e-6,\n        help=\"Learning rate (much lower than SFT!)\"\n    )\n    parser.add_argument(\n        \"--beta\", type=float, default=0.1,\n        help=\"KL-divergence penalty weight\"\n    )\n    \n    parser.add_argument(\n        \"--output_dir\",\n        default=\"./models/dpo_hallucination_resistant\",\n        help=\"Output directory for trained model\"\n    )\n    parser.add_argument(\n        \"--device\", default=\"cuda\",\n        help=\"Device to use\"\n    )\n    \n    args = parser.parse_args()\n    \n    config = DPOConfig(\n        sft_model_path=args.sft_model_path,\n        train_data_path=args.train_data_path,\n        val_data_path=args.val_data_path,\n        num_epochs=args.num_epochs,\n        batch_size=args.batch_size,\n        learning_rate=args.learning_rate,\n        beta=args.beta,\n        output_dir=args.output_dir,\n    )\n    \n    logger.info(\"DPO Training Configuration:\")\n    logger.info(json.dumps(vars(config), indent=2, default=str))\n    \n    trainer = DPOTrainer(config)\n    train_loader, val_loader = trainer.create_dataloaders()\n    stats = trainer.train(train_loader, val_loader)\n    \n    logger.info(\"\\nDPO Training completed successfully!\")\n    logger.info(f\"Model saved to {config.output_dir}\")\n\n\nif __name__ == \"__main__\":\n    main()\n"